# AutoCAD MegaIntellegence Project


In [1]:
import comtypes.client
import numpy as np
import pandas as pd
import math
import array
import time
from traceback import print_exc
app = comtypes.client.GetActiveObject('AutoCAD.Application')
activedoc = app.ActiveDocument
msp =activedoc.ModelSpace

In [2]:
# print all Block refs:
# for blk in activedoc.Blocks:
#     print(blk)

In [3]:
# set variables
df = pd.DataFrame(columns ={'device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y','first_point'})
df = df[['device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y','first_point']]
dim_aligned, dim_rotated, blockref, device_n = 0,0,0,0
# view each object in ModelSpace
for item in msp:
    #TODO: need to check if that actually device block
    #if it's a device:
    if item.ObjectName == 'AcDbBlockReference':
        #find BLOCK object using given BlockRef
        block = activedoc.Blocks.Item(item.Name)
        print (f'Block ref name: {block}')
        for element in block:
            if element.Layer =="attachment point":
                # fill the dataframe with the coordinates of the attachment points. Coordinates include distance from point (0,0,0) and insertion point
                  df = df.append({'device': device_n, \
                                'start_att_line_X': element.StartPoint[0] + item.InsertionPoint[0], \
                                'start_att_line_Y': element.StartPoint[1] + item.InsertionPoint[1], \
                                'end_att_line_X': element.EndPoint[0] + item.InsertionPoint[0],\
                                'end_att_line_Y': element.EndPoint[1] + item.InsertionPoint[1]},  ignore_index=True)
        device_n+= 1
df['device'] = df['device'].astype(int)

Block ref name: <POINTER(IAcadBlock) ptr=0x265f4b77f88 at 265f5fe5cc8>
Block ref name: <POINTER(IAcadBlock) ptr=0x265f4b78768 at 265f60dbec8>
Block ref name: <POINTER(IAcadBlock) ptr=0x265f1b342d8 at 265f6022ec8>


In [4]:
df

,device,start_att_line_X,end_att_line_X,start_att_line_Y,end_att_line_Y,first_point
0,0,468.643810,473.643810,569.910725,569.910725,NaN
1,0,471.143810,471.143810,572.410725,567.410725,NaN
2,0,538.643810,543.643810,569.910725,569.910725,NaN
3,0,541.143810,541.143810,572.410725,567.410725,NaN
4,0,468.643810,473.643810,539.910725,539.910725,NaN
5,0,471.143810,471.143810,542.410725,537.410725,NaN
6,0,538.643810,543.643810,539.910725,539.910725,NaN
7,0,541.143810,541.143810,542.410725,537.410725,NaN
8,1,656.799864,661.799864,539.910725,539.910725,NaN
9,1,656.799864,661.799864,569.910725,569.910725,NaN



Here we need to find center of attachment point. Attachment point is the center of the cross of attachment lines.
Length of attraction line must be 5mm.


In [5]:
# Is the line in this row horisontal?
df['horizontal']= (df.iloc[:,1] == df.iloc[:,2])
# Find center of this line at axis X. If line is horizontal...
df['point_X'] =np.where(df['horizontal'], \
                        # any coordinate is suitabledf
                        df.iloc[:, 1], \
                        # otherwise, take the middle of the line by X
                        (np.minimum(df.iloc[:, 1], df.iloc[:, 2]) +2.5))

#same for Y
df['point_Y'] =np.where(~ df['horizontal'], \
                        df.iloc[:, 3], \
                        (np.minimum(df.iloc[:, 3], df.iloc[:, 4]) +2.5)) 
# Make feature with coordinates of attachment point
df['first_point'] =df.point_X.astype(str) + ',' + df.point_Y.astype(str)


In [6]:
# Clear dataframe
df = df.iloc[:,[0,-2,-1]]
df = df.drop_duplicates()
df

,device,point_X,point_Y
0,0,471.143810,569.910725
2,0,541.143810,569.910725
4,0,471.143810,539.910725
6,0,541.143810,539.910725
8,1,659.299864,539.910725
9,1,659.299864,569.910725
10,2,475.100512,460.108155
11,2,545.100512,460.108155


----------
Now we need to build a Dataframe with diemensions.

In [7]:
# dims = []
# for item in msp:    
#     if item.ObjectName =='AcDbRotatedDimension':
#         print('found rotated dimention.')
#         print(item)
#         #print(f'Object layer name: {item.Layer}\n')       
#         dim_rotated = item
#     elif item.ObjectName == 'AcDbAlignedDimension':
#         print('found aligned dimention\n')
#         dim_aligned = item
#         dims.append(dim_aligned)   

ACAD is very unfriendly to Python developers. It took a while to connect python code to ACAD via 'win32.com' and 'comtypes'. But, this libs still can't make methods to extract all the data from all types of dimensions. So, I will extract drawning to .DXF file. This file contains all the data about each element of drawning.

-----------
Program export .dwg to .dxf here. Now we can find any property of any element of drawning.

In [8]:
sset = activedoc.SelectionSets.Add('SSET')

In [9]:
#Make DXF file to parse 
filename= 'C:/Users/MSI/Documents/AutoCad_Project/testfile'
activedoc.Export(filename, 'DXF', sset)
dxf_file = open(filename + '.dxf', 'r')
dxf = list(dxf_file)
dxf_file.close()

#we can cut \n if we need
#dxf = [st.rstrip() for st in dxf]


In [17]:
#Looking for lines in dxf file containing dimensions data
entities, dimensions = [],[]

for i, element in enumerate(dxf):
    if element == 'AcDbEntity\n': entities.append(i)
    elif element =='AcDbDimension\n': dimensions.append(i+1)

print(f'Objects at the drawning: {len(entities)}\n Dimenssions : {len(dimensions)}')

Objects at the drawning: 161
 Dimenssions : 8


In [19]:
dimensions

[6722, 6780, 6960, 7040, 7096, 7176, 7232, 7288]

In [20]:
df_dimensions = pd.DataFrame(columns ={'device','start_X', 'end_X',
                                       'start_Y', 'end_Y',
                                       'text_X', 'text_Y'})
df_dimensions = df_dimensions[['device','start_X', 'end_X',
                               'start_Y', 'end_Y',
                               'text_X','text_Y']]
l=['7','7','7','7','7','7']
l=pd.Series(l)
df_dimensions.append({'end_X':l}, ignore_index=True)

,device,start_X,end_X,start_Y,end_Y,text_X,text_Y
0,NaN,NaN,0 7 1 7 2 7 3 7 4 7 5 7 dtyp...,NaN,NaN,NaN,NaN


In [31]:
dd = df_dimensions
start_X, end_X, start_Y, end_Y, text_X, text_Y = '','','','','',''
try:
    for ent in dimensions:
        print('Dimension data:')
        for pos, line in enumerate(dxf[ent:]):
            if line == ' 13\n':
                start_X = float((dxf[ent+pos+1]).rstrip())
                print(f'1st point X : {dxf[ent+pos+1]}')
            elif line == ' 23\n':
                end_X = float((dxf[ent+pos+1]).rstrip())
                print(f'1st point Y : {dxf[ent+pos+1]}')
            elif line == ' 14\n':
                start_Y = float((dxf[ent+pos+1]).rstrip())
                print(f'2nd point X : {dxf[ent+pos+1]}')
            elif line == ' 24\n':
                end_Y = float((dxf[ent+pos+1]).rstrip())
                print(f'2nd point Y: {dxf[ent+pos+1]}')
            elif line == ' 11\n':
                text_X = float((dxf[ent+pos+1]).rstrip())
                print(f'Text coord X: {dxf[ent+pos+1]}')
            elif line == ' 21\n':
                text_Y = float((dxf[ent+pos+1]).rstrip())
                print(f'Text coord Y: {dxf[ent+pos+1]}')
            elif (dxf[ent+pos] == 'AcDbEntity\n') or (dxf[ent+pos] =='ENDSEC\n'):break
        dd = dd.append({'start_X':start_X, 'end_X':end_X,
                        'start_Y':start_Y, 'end_Y':end_Y,
                        'text_X':text_X, 'text_Y':text_Y}, ignore_index = True)
            
except Exception as e:
    print_exc()
dd

Dimension data:
Text coord X: 506.1438103262689

Text coord Y: 602.0422114926178

1st point X : 541.1438103262689

1st point Y : 569.9107245164855

2nd point X : 471.143810326269

2nd point Y: 569.9107245164855

Dimension data:
Text coord X: 450.0

Text coord Y: 550.0

1st point X : 471.143810326269

1st point Y : 569.9107245164855

2nd point X : 471.143810326269

2nd point Y: 539.9107245164855

Dimension data:
Text coord X: 635.3801847614122

Text coord Y: 708.7287361436827

1st point X : 591.007589752302

1st point Y : 673.992738365021

2nd point X : 679.7527797705225

2nd point Y: 673.992738365021

Dimension data:
Text coord X: 786.7885402455661

Text coord Y: 1000.0

1st point X : 500.0

1st point Y : 800.0

2nd point X : 500.0

2nd point Y: 1200.0

Dimension data:
Text coord X: 886.7885402455661

Text coord Y: 1000.0

1st point X : 500.0

1st point Y : 800.0

2nd point X : 500.0

2nd point Y: 1200.0

Dimension data:
Text coord X: 924.7307707394357

Text coord Y: 1000.0

1st point 

,device,start_X,end_X,start_Y,end_Y,text_X,text_Y
0,NaN,541.143810,569.910725,471.143810,569.910725,506.143810,602.042211
1,NaN,471.143810,569.910725,471.143810,539.910725,450.000000,550.000000
2,NaN,591.007590,673.992738,679.752780,673.992738,635.380185,708.728736
3,NaN,500.000000,800.000000,500.000000,1200.000000,786.788540,1000.000000
4,NaN,500.000000,800.000000,500.000000,1200.000000,886.788540,1000.000000
5,NaN,500.000000,800.000000,500.000000,1200.000000,924.730771,1000.000000
6,NaN,500.000000,800.000000,500.000000,1200.000000,486.788540,1000.000000
7,NaN,1056.458511,562.406336,1225.928421,733.305723,1595.265392,719.419487


In [14]:
print(dim_aligned.ExtLine1Point[0])
print(dim_aligned.ExtLine2Point)
print(dim_aligned.TextPosition)

AttributeError: 'int' object has no attribute 'ExtLine1Point'

In [ ]:
# Довольно неочевидно, как взрывать размеры, не используя vba или lisp. Здесь надо будет добыть имя блока размера для метода explode.
# for item in msp:
#     if item.ObjectName == 'AcAlignedDimension':
#         #find BLOCK object by given BlockRef
#         activedoc.Blocks.Item(item.     )
#         #item.Explode(item.ObjectID)

#         print (item.ObjectID) 

Использовать фичу DIMATFIT 

вариант добычи точек привязки rotated размера - экспорт в dwf https://adn-cis.org/forum/index.php?topic=8448.msg31681#msg31681
И не забыть SelectionSet

In [ ]:
def FindAttachmentPoint(data):
    """This function converts given data to the coordinates of attachment points. 
    
    data: dataframe with coordinates of lines in layer 'attachment point'
    
    return
    """
    #FindAttachmentPoint(df[df['device']==device])
    print(f'data:\n{data}')
    print(data.iloc[[0]])


def MakeDimVertical(msp, point1, point2, loc):
    """This function make vertical dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass
    

def MakeDimHorizontal(msp, point1, point2, loc):
    """This function make horizontal dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass

def ProcessAttachmentPoints(msp, objects):
    """This function make coordinates of attachment points.
    
    msp:  model space
    objects:  lines from specified layer
    """
    pass


In [ ]:
stop
# ActiveX usage examples
pt = array.array('d', [0,0,0]) # convert to variant
point = msp.AddPoint(pt)
# Add a LINE
pt1 = array.array('d', [0.0,0.0,0]) # start point
pt2 = array.array('d', [20.0,20.0,0]) # end point
line = msp.AddLine(pt1, pt2)

In [ ]:
s1 =  array.array('d', [500.0, 800.0, 0.0])
s2 =  array.array('d', [500.0, 1200.0, 0.0])
loc = array.array('d', [400.0, 1000.0, 0.0])
new_dim = msp.AddDimAligned(s1, s2, loc)
new_dim.update()

In [ ]:
df = pd.read_excel('E://Python/RubinProject/Journal.xlsx')

----------------------

This old code shows how to extract data from 'aligned' dimensions throw ActiveX.

In [ ]:
# df_device = df.rename(columns={'first_point':'point'})\
#             .drop(columns=['start_att_line_X','end_att_line_X','start_att_line_Y','end_att_line_Y',\
#                            'point_X','point_Y', 'horizontal']).drop_duplicates(subset= 'point')

# df_dimensions = pd.DataFrame()
# #make the dataframe with dimensions data
# for i, dim in enumerate(dims):
#     df_dimensions = df_dimensions.append({'dim_No': i+1 , \
#                               'start_X': dim.ExtLine1Point[0],
#                               'start_Y': dim.ExtLine1Point[1],
#                               'end_X': dim.ExtLine2Point[0],
#                               'end_Y': dim.ExtLine2Point[1], 
#                               'text_pos_X': dim.TextPosition[0], 
#                               'text_pos_Y': dim.TextPosition[1],
#                               'first_point': f'{dim.ExtLine1Point[0]},{dim.ExtLine1Point[1]}',
#                               'second_point': f'{dim.ExtLine2Point[0]},{dim.ExtLine2Point[1]}'}, ignore_index=True)

# df_dimensions = df_dimensions[['dim_No','start_X','start_Y','end_X','end_Y','text_pos_X','text_pos_Y','first_point','second_point']]

# # process dataframe
# df4 = df_dimensions.drop(columns = ['start_X','start_Y','end_X','end_Y'])
# df2 = pd.concat([(df4.drop(columns='second_point')),(df4.drop(columns='first_point'))],sort=False)
# # make 'point' feature. Make 2 rows for each dim_No. First row stores first coordinate on 1st place. Second row stores 2nd coordinate on 1st place
# points = pd.concat([df2.iloc[:,3],df2.iloc[:,4]]).dropna()
# points.name = 'point'
# df_dimensions = pd.concat([df2, points], axis=1) \
#                     .drop(columns=['first_point', 'second_point'])

# df_dimensions

# # df2

# # df4 = df_dimensions.drop(columns = {'start_X','start_Y','end_X','end_Y'})
# # pd.merge((df4.drop(columns='second_point')),\
# #           (df4.drop(columns='first_point')), \
# #           how ='outer', \
# #           left_on= 'first_point', \
# #           right_on = 'second_point')

In [ ]:
# df= pd.merge(left = df_device, right = df_dimensions, how= 'outer', on='point')
# df= df.fillna(0)
# df['dim_No'] = df['dim_No'].astype(int)
# df

In [ ]:
#activedoc.SendCommand("(cdr (assoc 13 (entget (handent " & """" & returnObj.Handle & """" & "))))" & vbCr)

-------------------

Graph algorithms

In [ ]:
# import heapq
# from sys import stdin, stdout
 
# # Dijktra's shortest path algorithm. Prints the path from source to target.


#def dijkstra(adj, source, target):
#     INF = ((1<<63) - 1)//2
#     pred = { x:x for x in adj }
#     dist = { x:INF for x in adj }
#     dist[ source ] = 0
#     PQ = []
#     heapq.heappush(PQ, [dist[ source ], source])
 
#     while(PQ):
#         u = heapq.heappop(PQ)  # u is a tuple [u_dist, u_id]
#         u_dist = u[0]
#         u_id = u[1]
#         if u_dist == dist[u_id]:
#             #if u_id == target:
#             #    break
#             for v in adj[u_id]:
#                v_id = v[0]
#                w_uv = v[1]
#                if dist[u_id] +  w_uv < dist[v_id]:
#                    dist[v_id] = dist[u_id] + w_uv
#                    heapq.heappush(PQ, [dist[v_id], v_id])
#                    pred[v_id] = u_id
                
#     if dist[target]==INF:
#         stdout.write("There is no path between ", source, "and", target)
#     else:
#         st = []
#         node = target
#         while(True):
#             st.append(str(node))
#             if(node==pred[node]):
#                 break
#             node = pred[node]
#         path = st[::-1]
#         stdout.write("The shortest path is: " + " ".join(path) + "\n\n")
#         stdout.write("The distance from 'a' to 'i' is: " + str(dist['i']) + "\n\n")
#         stdout.write("distance dictionary: " + str(dist) + "\n\n")
#         stdout.write("predecessor dictionary: " + str(pred))
     
# #----------------------------------------------------------
 
# def main():
     
#     adj = {'c': [('b', 0.32), ('e', 0.17), ('f', 0.91)],
#          'g': [('d', 0.17), ('e', 0.27), ('h', 0.92)],
#          'i': [('e', 1.98), ('f', 0.13), ('h', 0.22)],
#          'f': [('c', 0.91), ('e', 0.33), ('i', 0.13)],
#          'h': [('e', 0.18), ('g', 0.92), ('i', 0.22)],
#          'd': [('a', 0.72), ('e', 0.29), ('g', 0.17)],
#          'a': [('b', 0.95), ('d', 0.72), ('e', 1.75)],
#          'e': [('a', 1.75), ('b', 0.82), ('c', 0.17), ('d', 0.29), ('f', 0.33), ('g', 0.27), ('h', 0.18), ('i', 1.98)],
#          'b': [('a', 0.95), ('c', 0.32), ('e', 0.82)]}
         
#     dijkstra(adj, 'a', 'i')
 
# #----------------------------------------------------------
 
# if __name__ == "__main__":
#     main()

In [ ]:
# def bellman_ford(graph, source):
#     # Step 1: Prepare the distance and predecessor for each node
#     distance, predecessor = dict(), dict()
#     for node in graph:
#         distance[node], predecessor[node] = float('inf'), None
#     distance[source] = 0

#     # Step 2: Relax the edges
#     for _ in range(len(graph) - 1):
#         for node in graph:
#             for neighbour in graph[node]:
#                 # If the distance between the node and the neighbour is lower than the current, store it
#                 if distance[neighbour] > distance[node] + graph[node][neighbour]:
#                     distance[neighbour], predecessor[neighbour] = distance[node] + graph[node][neighbour], node

#     # Step 3: Check for negative weight cycles
#     for node in graph:
#         for neighbour in graph[node]:
#             assert distance[neighbour] <= distance[node] + graph[node][neighbour], "Negative weight cycle."
 
#     return distance, predecessor
    
# if __name__ == '__main__':
#     graph = {
#         'a': {'b': -1, 'c':  4},
#         'b': {'c':  3, 'd':  2, 'e':  2},
#         'c': {},
#         'd': {'b':  1, 'c':  5},
#         'e': {'d': -3}
#     }

#     distance, predecessor = bellman_ford(graph, source='a')

#     print(distance)

In [ ]:
# import win32com.client
# acad = win32com.client.Dispatch("AutoCAD.Application")

# doc = acad.ActiveDocument   # Document object


# # iterate trough all objects (entities) in the currently opened drawing
# # and if its a BlockReference, display its attributes and some other things.
# for entity in acad.ActiveDocument.ModelSpace:
#     name = entity.EntityName
#     if name == 'AcDbBlockReference':
#         HasAttributes = entity.HasAttributes
#         if HasAttributes:
#             print(entity.Name)
#             print(entity.Layer)
#             print(entity.ObjectID)
#             for attrib in entity.GetAttributes():
#                 print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
#                 # update text
#                 attrib.TextString = 'modified with python'
#                 attrib.Update()

In [ ]:
     
    #explode blocks
#    elif item.ObjectName == 'AcDbBlockReference':       
#         try:
#             print(f'Found a block.\n')
#             block = item
#             block.Explode()
#         except:
#             pass    # here we have some VARIANT bug. So just proceed.
#         i+=1

Фичи размеров в качестве таргетов:
- TextPosition (rotated размеры тоже их имеют)

todo:

точка крепления может быть не посередине линии соответствующего слоя, а где-то на пересечении с линией прибора. Сделать проверку на небольшое отклонение и выдать сообщение чтобы поправили чертёж

Как взрывать rotated размеры для извлечения координат точек привязки размеров - неочевидно. Даже непонятно зачем.